In [18]:
# !pip install emoji==2.10.0
# !pip install pymorphy2==0.9.1
# !pip install emojis==0.7.0
# !pip install telebot==0.0.5
# !pip install python-dotenv==1.0.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [19]:
# !pip show emoji
# !pip show pymorphy2
# !pip show emojis
# !pip show telebot
# !pip show python-dotenv

Name: python-dotenv
Version: 1.0.1
Summary: Read key-value pairs from a .env file and set them as environment variables
Home-page: https://github.com/theskumar/python-dotenv
Author: Saurabh Kumar
Author-email: me+github@saurabh-kumar.com
License: BSD-3-Clause
Location: c:\users\user\anaconda3\envs\gpu\lib\site-packages
Requires: 
Required-by: 


# Шаг 1: сбор датасета для проекта

In [ ]:
# !pip install selenium==4.17.2

Web драйвер для браузера был скачен с сайта https://googlechromelabs.github.io/chrome-for-testing/ (версия 121.0.6167.161)

In [13]:
from selenium import webdriver # для браузера
from selenium.webdriver.chrome.service import Service # сервис для запуска Chrome 
from selenium.webdriver.common.keys import Keys # для отправки запросов
from selenium.webdriver.common.by import By # для поиска на странице
import time # для 
import re # регулярные выражения для работы с текстом
import json # для сохранения файлов

In [14]:
executable_path = './chromedriver.exe' # путь до запускаемого драйвера
service = Service(executable_path=executable_path) # настройки для запуска браузера
pages = 5 # число страниц для рассмотрения

In [ ]:
movies = [] # список под фильмы в формате
# [{
#   "title": название фильма, 
#   "description": описание, 
#   "emojis": транскрипция в эмодзи,
#   "categories": категории, 
#   "year": год выхода,
#   "directors": режиссёры, 
#   "actors": известные актёры, 
#   "tagline": слоган,
#   "hint": подсказка про наличие цифр в названии,
#   "movie_url": ссылка на страницу фильма
#  }, ...
# ]

Сначала соберём основную информацию о мультфильме (название + ссылка + подсказки для названия).

In [ ]:
driver = webdriver.Chrome(service=service) # запуск браузера
driver.get("https://www.kinopoisk.ru/") # открытие страницы (GET запрос)
time.sleep(15) # время для введения капчи

for page in range(1, pages+1): # идём по числу страниц
    url = f"http://www.kinopoisk.ru/lists/movies/genre--animation/?b=films&sort=votes&page={page}" # ссылка с указанием страницы
    driver.get(url) # открытие страницы (GET запрос)

    # for movie in driver.find_elements("class name", "styles_root__ti07r"): # находим все элементы, отвечающие за мультфильмы на сайте и проходим по им всем
    for movie in driver.find_elements(By.CLASS_NAME, "styles_root__ti07r"): # находим все элементы, отвечающие за мультфильмы на сайте и проходим по им всем
        try: # так как может попасться рекламный блок
            title = movie.find_elements(By.TAG_NAME, "span")[0].text # находим название фильма (первый span элемент)
        except:
            continue # переходим к следующему элементу

        hint = [] # подсказка про наличие чисел в названии
        for word in re.sub('[^\w\s]', '', title).split(): # идём по "словам" в названии, в котором удалено всё, что не(^) буква/цифра/_(\w) и не пробельный символ(\s)
            if word.isdigit(): # если попали на число в названии
                hint.append(word) # добавляем "слово" (число) в список для подсказок
        
        movie_url = movie.find_element(By.TAG_NAME, "a").get_attribute("href") # получение ссылки на фильм (например: 'https://www.kinopoisk.ru/film/775276/')
        
        movies.append({"title": title, "description": None, "emojis": None, "categories": None, "year": None, "directors": None, "actors": None, "tagline": None, "hint": hint, "movie_url": movie_url}) # добавляем фильм в список с фильмами в виде словаря
driver.quit() # закрытие браузера

In [ ]:
movies[15] # пример содержимого списка фильмов после первого прохода

{'title': 'Шрэк 2',
 'description': None,
 'emojis': None,
 'categories': None,
 'year': None,
 'directors': None,
 'actors': None,
 'tagline': None,
 'hint': ['2'],
 'movie_url': 'https://www.kinopoisk.ru/film/5273/'}

Затем пройдём по полученным на первом шаге ссылкам и возьмём подробную информацию о мультфильмах.

In [ ]:
driver = webdriver.Chrome(service=service) # запуск браузера

driver.get("https://www.kinopoisk.ru/") # открытие страницы (GET запрос)
time.sleep(15) # время для введения капчи

for movie in movies: # идём по списку фильмов
    driver.get(movie["movie_url"]) # открытие страницы фильма (GET запрос)

    movie["year"] = driver.find_element(By.XPATH, "//*[contains(text(), 'Год производства')]").find_element(By.XPATH, "following-sibling::*").text # получаем год

    movie["tagline"] = driver.find_element(By.XPATH, "//*[contains(text(), 'Слоган')]").find_element(By.XPATH, "following-sibling::*").text # получаем слоган

    movie["description"] = "" # синопсис
    for p in driver.find_elements(By.CLASS_NAME, "styles_paragraph__wEGPz"): # находим блоки с текстом <p>...</p>
        movie["description"] += p.text + " " # добавляем их содержимое в синопсис
    for p in driver.find_elements(By.CLASS_NAME, "styles_root__aZJRN"): # находим блоки с текстом <p>...</p>
        movie["description"] += p.text # добавляем их содержимое в синопсис

    movie["categories"] = [] # категории
    for category in driver.find_element(By.XPATH, "//*[contains(text(), 'Жанр')]").find_element(By.XPATH, "following-sibling::*").find_elements(By.TAG_NAME, "a"): # находим все элементы, соответствующие категориям (оформлены в виде ссылок)
        movie["categories"].append(category.text) # добавляем категорию
    
    movie["directors"] = [] # режиссёр
    for director in driver.find_element(By.XPATH, "//*[contains(text(), 'Режиссер')]").find_element(By.XPATH, "following-sibling::*").find_elements(By.TAG_NAME, "a"): # находим все элементы, соответствующие режиссёрам (оформлены в виде ссылок)
        movie["directors"].append(director.text) # добавляем режиссёра

    movie["actors"] = [] # известные актёры
    try: # в некоторых мультфильмов нет списка актёров...
        for actor in driver.find_elements(By.CLASS_NAME, "styles_list___ufg4")[0].find_elements(By.TAG_NAME, "li"): # находим первый (0) список ul, он содержит известных актёров и берём из него все элементы
            movie["actors"].append(actor.text) # добавляем текст поля li (актёра) в список
    except: # пусть так и останется
        pass

driver.quit() # закрытие браузера

In [ ]:
movies[15] # пример содержимого списка мультфильмов после второго прохода

{'title': 'Шрэк 2',
 'description': 'Шрэк и Фиона возвращаются после медового месяца и находят письмо от родителей Фионы с приглашением на ужин. Однако те не подозревают, что их дочь тоже стала огром! Вместе с Осликом счастливая пара отправляется в путешествие, полное неожиданностей, и попадает в круговорот событий, во время которых приобретает множество друзей… ',
 'emojis': None,
 'categories': ['мультфильм',
  'фэнтези',
  'мелодрама',
  'комедия',
  'приключения',
  'семейный',
  'слова'],
 'year': '2004',
 'directors': ['Эндрю Адамсон', 'Келли Эсбёри', 'Конрад Вернон'],
 'actors': ['Майк Майерс',
  'Эдди Мерфи',
  'Кэмерон Диас',
  'Джули Эндрюс',
  'Антонио Бандерас',
  'Джон Клиз',
  'Руперт Эверетт',
  'Дженнифер Сондерс',
  'Арон Уорнер',
  'Келли Эсбёри'],
 'tagline': '«Not so far, far away...»',
 'hint': ['2'],
 'movie_url': 'https://www.kinopoisk.ru/film/5273/'}

In [17]:
with open('./data/movies.json', 'w') as f: # открываем файл для записи ('w')
    json.dump(movies, f) # сохраняем в него данные о фильмах

# Шаг 2: модель для суммаризации текста

<b>
Цель: найти модель, что наилучшим образом (быстро и качественно) проводит суммаризацию синопсиса фильма и провести её fine-tuning. 
</b>

In [ ]:
with open('./data/movies.json', 'r') as f: # открываем файл для чтения ('r')
    movies = json.load(f)

# Шаг 3: конвертация текста в эмодзи

<b>
Цель: рассмотреть варианты конвертации полученного саммари в последовательность эмодзи. 
</b>

### Вариант 1: использование специально обученной модели

In [ ]:
# !pip install text_unidecode
# pytorch 
# scikit-learn
# text-unidecode
# emoji

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 78.2/78.2 kB 2.2 MB/s eta 0:00:00


In [ ]:
ROOT_PATH = "./models/converting"
VOCAB_PATH = f'{ROOT_PATH}/vocabulary.json'
PRETRAINED_PATH = f'{ROOT_PATH}/torchMoji.bin'

In [ ]:
import sys
import json
import emoji
import numpy as np

# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append(ROOT_PATH)

In [ ]:
sys.path

['c:\\Users\\User\\Desktop\\документы\\учёба\\магистратура (ИАД 2023-2025)\\курс 1\\5) НИС - Инструменты интеллектуального анализа данных\\pet_project',
 'c:\\Users\\User\\anaconda3\\envs\\gpu\\python39.zip',
 'c:\\Users\\User\\anaconda3\\envs\\gpu\\DLLs',
 'c:\\Users\\User\\anaconda3\\envs\\gpu\\lib',
 'c:\\Users\\User\\anaconda3\\envs\\gpu',
 '',
 'C:\\Users\\User\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'c:\\Users\\User\\anaconda3\\envs\\gpu\\lib\\site-packages',
 'c:\\Users\\User\\anaconda3\\envs\\gpu\\lib\\site-packages\\win32',
 'c:\\Users\\User\\anaconda3\\envs\\gpu\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\User\\anaconda3\\envs\\gpu\\lib\\site-packages\\Pythonwin',
 './models/converting',
 './models/converting',
 './models/converting']

In [ ]:
# import models.converting.torchmoji as torchmoji
from models.converting.torchmoji.sentence_tokenizer import SentenceTokenizer
from models.converting.torchmoji.model_def import torchmoji_emojis

In [ ]:
# Loading model
model = torchmoji_emojis(PRETRAINED_PATH)
model

c:\Users\User\Desktop\документы\учёба\магистратура (ИАД 2023-2025)\курс 1\5) НИС - Инструменты интеллектуального анализа данных\pet_project\models\converting\torchmoji\model_def.py:159: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.embed.weight.data, a=-0.5, b=0.5)
c:\Users\User\Desktop\документы\учёба\магистратура (ИАД 2023-2025)\курс 1\5) НИС - Инструменты интеллектуального анализа данных\pet_project\models\converting\torchmoji\model_def.py:161: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(t)
c:\Users\User\Desktop\документы\учёба\магистратура (ИАД 2023-2025)\курс 1\5) НИС - Инструменты интеллектуального анализа данных\pet_project\models\converting\torchmoji\model_def.py:163: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  nn.init.orthogonal(t)
c:\Users\User\Desktop\документы\учёба\магистратура (ИАД 2023-2025)\курс 1\5) НИС -

TorchMoji(
  (embed): Embedding(50000, 256)
  (embed_dropout): Dropout2d(p=0, inplace=False)
  (lstm_0): LSTMHardSigmoid(256, 512, batch_first=True, bidirectional=True)
  (lstm_1): LSTMHardSigmoid(1024, 512, batch_first=True, bidirectional=True)
  (attention_layer): Attention(2304, return attention=False)
  (final_dropout): Dropout(p=0, inplace=False)
  (output_layer): Sequential(
    (0): Linear(in_features=2304, out_features=64, bias=True)
    (1): Softmax(dim=None)
  )
)

In [ ]:
with open(VOCAB_PATH, 'r') as f:
    vocabulary = json.load(f)
list(vocabulary.items())[:20]

[('CUSTOM_MASK', 0),
 ('CUSTOM_UNKNOWN', 1),
 ('CUSTOM_AT', 2),
 ('CUSTOM_URL', 3),
 ('CUSTOM_NUMBER', 4),
 ('CUSTOM_BREAK', 5),
 ('CUSTOM_BLANK_6', 6),
 ('CUSTOM_BLANK_7', 7),
 ('CUSTOM_BLANK_8', 8),
 ('CUSTOM_BLANK_9', 9),
 ('the', 10),
 ('.', 11),
 ('and', 12),
 ('you', 13),
 (',', 14),
 ('a', 15),
 ('for', 16),
 ('to', 17),
 ('i', 18),
 ('!', 19)]

In [ ]:
# texts = ['Пример текста для анализа эмоций']
texts = ["A tale set on death row in a Southern jail, where gentle giant John Coffee (Michael Clarke Duncan) possesses the mysterious power to heal people's ailments. When the lead guard, Paul Edgecombe (Tom Hanks), recognizes John's gift, he tries to help stave off the condemned man's execution."]

sentence_tokenizer = SentenceTokenizer(vocabulary, 100)

# tokenized, _, _ = sentence_tokenizer.tokenize_sentences(texts)
tokenized, _, _ = sentence_tokenizer.tokenize_sentences(texts)
tokenized

array([[   15,  2115,   389,    44,   748,  2323,    36,    15,  3393,
         2371,    14,   176,  6582,  2169,   858,   511,    99,  1532,
        17189,  9542,    97,     1,    10,  7841,   526,    17,  6774,
         2405,     1,    11,    63,    10,  1802,  3467,    14,  1209,
            1,    99,  1820, 19736,    97,    14, 13727, 12895,   580,
           14,   189,  2585,    17,   340,     1,   109,    10, 43549,
         3744,  3941,    11,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0]], dtype=uint16)

In [ ]:
# Get sentence probability
prob = model(tokenized)[0]
prob

c:\Users\User\anaconda3\envs\gpu\lib\site-packages\torch\nn\functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
c:\Users\User\anaconda3\envs\gpu\lib\site-packages\torch\nn\functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
c:\Users\User\anaconda3\envs\gpu\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


array([0.00826604, 0.00794725, 0.00388433, 0.00444268, 0.00250981,
       0.01025149, 0.00719884, 0.0085858 , 0.01724289, 0.00555669,
       0.00797884, 0.02316808, 0.02480408, 0.00709422, 0.00218912,
       0.00350395, 0.00767403, 0.01006989, 0.00342063, 0.02203122,
       0.01141116, 0.02512378, 0.03316531, 0.00193796, 0.02461221,
       0.032507  , 0.00186689, 0.02046843, 0.00663869, 0.00528804,
       0.01512323, 0.0126198 , 0.0813021 , 0.02874656, 0.03152588,
       0.00842565, 0.00332252, 0.02005173, 0.00240824, 0.00955561,
       0.01205385, 0.01638797, 0.01766201, 0.02012684, 0.03690895,
       0.01790947, 0.02698752, 0.01898514, 0.00781498, 0.00443472,
       0.02008365, 0.02816985, 0.02341518, 0.01876715, 0.02068366,
       0.05879206, 0.00465287, 0.01031846, 0.01625735, 0.00929255,
       0.00573174, 0.01060102, 0.02345336, 0.00658895], dtype=float32)

In [ ]:
EMOJIS = ":joy: :unamused: :weary: :sob: :heart_eyes: \
:pensive: :ok_hand: :blush: :heart: :smirk: \
:grin: :notes: :flushed: :100: :sleeping: \
:relieved: :relaxed: :raised_hands: :two_hearts: :expressionless: \
:sweat_smile: :pray: :confused: :kissing_heart: :heartbeat: \
:neutral_face: :information_desk_person: :disappointed: :see_no_evil: :tired_face: \
:v: :sunglasses: :rage: :thumbsup: :cry: \
:sleepy: :yum: :triumph: :hand: :mask: \
:clap: :eyes: :gun: :persevere: :smiling_imp: \
:sweat: :broken_heart: :yellow_heart: :musical_note: :speak_no_evil: \
:wink: :skull: :confounded: :smile: :stuck_out_tongue_winking_eye: \
:angry: :no_good: :muscle: :facepunch: :purple_heart: \
:sparkling_heart: :blue_heart: :grimacing: :sparkles:".split(' ')

In [ ]:
def top_elements(array, k):
    ind = np.argpartition(array, -k)[-k:]
    return ind[np.argsort(array[ind])][::-1]

# Top emoji id
emoji_ids = top_elements(prob, 20)

# map to emojis
emojis = map(lambda x: EMOJIS[x], emoji_ids)

print(emoji.emojize("{} {}".format(texts[0],' '.join(emojis)), language='alias'))

A tale set on death row in a Southern jail, where gentle giant John Coffee (Michael Clarke Duncan) possesses the mysterious power to heal people's ailments. When the lead guard, Paul Edgecombe (Tom Hanks), recognizes John's gift, he tries to help stave off the condemned man's execution. 😡 😠 😈 😕 😐 😢 👍 💀 💔 🙏 😳 💓 😬 😖 🎶 😑 😜 😞 😣 😉


### Вариант 2: ручное преобразование текста в эмодзи

In [ ]:
# import pymorphy2
# morph = pymorphy2.MorphAnalyzer()
# morph.parse('мировой войны')

In [ ]:
# import pymorphy2

# # Инициализация анализатора
# morph = pymorphy2.MorphAnalyzer()

# # Слово для лемматизации
# word = "бегущий"

# # Лемматизация слова
# parsed_word = morph.parse(word)[0]
# lemma = parsed_word.normal_form

# # Вывод результата
# print(lemma)

In [ ]:
import emoji

# Конвертация текста в эмодзи
# emoji_text = emoji.emojize(input_text, language='ru', use_aliases=True)
emoji_text = emoji.emojize(input_text, language='alias')
# emoji_text = emoji.emojize('Python is :thumbs_up:', use_aliases=True)

# Вывод результата
print(emoji_text)

NameError: name 'input_text' is not defined

In [ ]:
# emoji.encode(":a:")
emoji.emojize(":A_button_(blood_type):", language="alias")

'🅰️'

In [ ]:
emoji.EMOJI_DATA

{'🥇': {'en': ':1st_place_medal:',
  'status': 2,
  'E': 3,
  'de': ':goldmedaille:',
  'es': ':medalla_de_oro:',
  'fr': ':médaille_d’or:',
  'ja': ':金メダル:',
  'ko': ':금메달:',
  'pt': ':medalha_de_ouro:',
  'it': ':medaglia_d’oro:',
  'fa': ':مدال_طلا:',
  'id': ':medali_emas:',
  'zh': ':金牌:',
  'ru': ':золотая_медаль:',
  'tr': ':birincilik_madalyası:',
  'ar': ':ميدالية_مركز_أول:'},
 '🥈': {'en': ':2nd_place_medal:',
  'status': 2,
  'E': 3,
  'de': ':silbermedaille:',
  'es': ':medalla_de_plata:',
  'fr': ':médaille_d’argent:',
  'ja': ':銀メダル:',
  'ko': ':은메달:',
  'pt': ':medalha_de_prata:',
  'it': ':medaglia_d’argento:',
  'fa': ':مدال_نقره:',
  'id': ':medali_perak:',
  'zh': ':银牌:',
  'ru': ':серебряная_медаль:',
  'tr': ':ikincilik_madalyası:',
  'ar': ':ميدالية_مركز_ثان:'},
 '🥉': {'en': ':3rd_place_medal:',
  'status': 2,
  'E': 3,
  'de': ':bronzemedaille:',
  'es': ':medalla_de_bronce:',
  'fr': ':médaille_de_bronze:',
  'ja': ':銅メダル:',
  'ko': ':동메달:',
  'pt': ':medalha_de_b

In [ ]:
# unique_aliases = [] # список со всеми уникальными псевдонимами эмодзи
# for e in emoji.EMOJI_DATA.keys(): # идём по эмодзи
#     if 'alias' in emoji.EMOJI_DATA[e]: # если у эмодзи есть псевдонимы
#         emoji.EMOJI_DATA[e]['alias'].append(emoji.EMOJI_DATA[e]['en']) # добавляем к псевдонимам само название эмодзи (в формате ":emoji_name:")
#     else: # иначе
#         emoji.EMOJI_DATA[e]['alias'] = [emoji.EMOJI_DATA[e]['en']] # название эмодзи становится его псевдонимом (в дальнейшем работа будет только с псевдонимами и variant?)
        
#     for drop_key in ['en', 'status', 'E', 'de', 'es', 'fr', 'ja', 'ko', 'pt', 'it', 'fa', 'id', 'zh', 'ru', 'tr', 'ar']: # идём по ключам данных исходного словаря эмодзи, что нам не потребуются 
#         del emoji.EMOJI_DATA[e][drop_key] # удаляем их
    
#     for alias in emoji.EMOJI_DATA[e]['alias']: # проходим по всем псевдонимам
#         if alias not in unique_aliases: # если такой псевдоним ещё не встречался
#             unique_aliases.append(alias) # запоминаем псевдоним эмодзи 

# emoji.EMOJI_DATA

{'🥇': {'alias': [':1st_place_medal:']},
 '🥈': {'alias': [':2nd_place_medal:']},
 '🥉': {'alias': [':3rd_place_medal:']},
 '🆎': {'alias': [':ab:',
   ':ab_button_blood_type:',
   ':AB_button_(blood_type):']},
 '🏧': {'alias': [':atm:', ':atm_sign:', ':ATM_sign:']},
 '🅰️': {'alias': [':a:', ':a_button_blood_type:', ':A_button_(blood_type):'],
  'variant': True},
 '🅰': {'alias': [':a:', ':a_button_blood_type:', ':A_button_(blood_type):'],
  'variant': True},
 '🇦🇫': {'alias': [':flag_for_Afghanistan:', ':afghanistan:', ':Afghanistan:']},
 '🇦🇱': {'alias': [':flag_for_Albania:', ':albania:', ':Albania:']},
 '🇩🇿': {'alias': [':flag_for_Algeria:', ':algeria:', ':Algeria:']},
 '🇦🇸': {'alias': [':flag_for_American_Samoa:',
   ':american_samoa:',
   ':American_Samoa:']},
 '🇦🇩': {'alias': [':flag_for_Andorra:', ':andorra:', ':Andorra:']},
 '🇦🇴': {'alias': [':flag_for_Angola:', ':angola:', ':Angola:']},
 '🇦🇮': {'alias': [':flag_for_Anguilla:', ':anguilla:', ':Anguilla:']},
 '🇦🇶': {'alias': [':flag_fo

In [ ]:
for a in unique_aliases:
    print(a)

:1st_place_medal:
:2nd_place_medal:
:3rd_place_medal:
:ab:
:ab_button_blood_type:
:AB_button_(blood_type):
:atm:
:atm_sign:
:ATM_sign:
:a:
:a_button_blood_type:
:A_button_(blood_type):
:flag_for_Afghanistan:
:afghanistan:
:Afghanistan:
:flag_for_Albania:
:albania:
:Albania:
:flag_for_Algeria:
:algeria:
:Algeria:
:flag_for_American_Samoa:
:american_samoa:
:American_Samoa:
:flag_for_Andorra:
:andorra:
:Andorra:
:flag_for_Angola:
:angola:
:Angola:
:flag_for_Anguilla:
:anguilla:
:Anguilla:
:flag_for_Antarctica:
:antarctica:
:Antarctica:
:flag_for_Antigua_&_Barbuda:
:antigua_barbuda:
:Antigua_&_Barbuda:
:aquarius:
:Aquarius:
:flag_for_Argentina:
:argentina:
:Argentina:
:aries:
:Aries:
:flag_for_Armenia:
:armenia:
:Armenia:
:flag_for_Aruba:
:aruba:
:Aruba:
:flag_for_Ascension_Island:
:ascension_island:
:Ascension_Island:
:flag_for_Australia:
:australia:
:Australia:
:flag_for_Austria:
:austria:
:Austria:
:flag_for_Azerbaijan:
:azerbaijan:
:Azerbaijan:
:back:
:back_arrow:
:BACK_arrow:
:b:
:b_b

<b> 
Библиотека emoji содержит все эмодзи из emojis, однако во второй имеются теги, что могут быть использованы как дополнительные псевдонимы. 
</b>

In [ ]:
import emojis
for e in emojis.db.get_emoji_aliases():
    print(e)

:grinning:
:smiley:
:smile:
:grin:
:laughing:
:satisfied:
:sweat_smile:
:rofl:
:joy:
:slightly_smiling_face:
:upside_down_face:
:melting_face:
:wink:
:blush:
:innocent:
:smiling_face_with_three_hearts:
:heart_eyes:
:star_struck:
:kissing_heart:
:kissing:
:relaxed:
:kissing_closed_eyes:
:kissing_smiling_eyes:
:smiling_face_with_tear:
:yum:
:stuck_out_tongue:
:stuck_out_tongue_winking_eye:
:zany_face:
:stuck_out_tongue_closed_eyes:
:money_mouth_face:
:hugs:
:hand_over_mouth:
:face_with_open_eyes_and_hand_over_mouth:
:face_with_peeking_eye:
:shushing_face:
:thinking:
:saluting_face:
:zipper_mouth_face:
:raised_eyebrow:
:neutral_face:
:expressionless:
:no_mouth:
:dotted_line_face:
:face_in_clouds:
:smirk:
:unamused:
:roll_eyes:
:grimacing:
:face_exhaling:
:lying_face:
:relieved:
:pensive:
:sleepy:
:drooling_face:
:sleeping:
:mask:
:face_with_thermometer:
:face_with_head_bandage:
:nauseated_face:
:vomiting_face:
:sneezing_face:
:hot_face:
:cold_face:
:woozy_face:
:dizzy_face:
:face_with_spi

In [ ]:
emojis.db.get_emoji_by_alias('trophy')

Emoji(aliases=['trophy'], emoji='🏆', tags=['award', 'contest', 'winner'], category='Activities', unicode_version='6.0')

In [ ]:
# emojis.db.get_emoji_by_alias("smiley")
for i in emojis.db.get_emojis_by_tag("winner"):
    print(i)

Emoji(aliases=['trophy'], emoji='🏆', tags=['award', 'contest', 'winner'], category='Activities', unicode_version='6.0')
Emoji(aliases=['medal_sports'], emoji='🏅', tags=['gold', 'winner'], category='Activities', unicode_version='7.0')


In [ ]:
emojis.db.get_categories()

{'Activities',
 'Animals & Nature',
 'Flags',
 'Food & Drink',
 'Objects',
 'People & Body',
 'Smileys & Emotion',
 'Symbols',
 'Travel & Places'}

In [ ]:
for category in emojis.db.get_categories():
    for emoji in emojis.db.get_emojis_by_category(category=category):
        print(emoji)

Emoji(aliases=['checkered_flag'], emoji='🏁', tags=['milestone', 'finish'], category='Flags', unicode_version='6.0')
Emoji(aliases=['triangular_flag_on_post'], emoji='🚩', tags=[], category='Flags', unicode_version='6.0')
Emoji(aliases=['crossed_flags'], emoji='🎌', tags=[], category='Flags', unicode_version='6.0')
Emoji(aliases=['black_flag'], emoji='🏴', tags=[], category='Flags', unicode_version='7.0')
Emoji(aliases=['white_flag'], emoji='🏳️', tags=[], category='Flags', unicode_version='7.0')
Emoji(aliases=['rainbow_flag'], emoji='🏳️\u200d🌈', tags=['pride'], category='Flags', unicode_version='6.0')
Emoji(aliases=['transgender_flag'], emoji='🏳️\u200d⚧️', tags=[], category='Flags', unicode_version='13.0')
Emoji(aliases=['pirate_flag'], emoji='🏴\u200d☠️', tags=[], category='Flags', unicode_version='11.0')
Emoji(aliases=['ascension_island'], emoji='🇦🇨', tags=[], category='Flags', unicode_version='11.0')
Emoji(aliases=['andorra'], emoji='🇦🇩', tags=[], category='Flags', unicode_version='6.0')

In [ ]:
import emoji

# Создание словаря соответствий слов и эмодзи
word_to_emoji = {
    "happy": ":smile:",
    "sad": ":cry:",
    # Добавьте другие соответствия слов и эмодзи по вашему усмотрению
}

# Ввод текста для конвертации
input_text = "I am happy and sad ."

# Разделение текста на отдельные слова
words = input_text.split()

# Конвертация каждого слова в соответствующее ему эмодзи
emoji_text = " ".join([word_to_emoji.get(word, word) for word in words])

# Вывод результата
print(emoji.emojize(emoji_text, language='alias'))

I am 😄 and 😢 .


### Вариант 3: использование word2vec модели для преобразования слов